Let us use Newton's method on the system defined by the function

In [ ]:
function nlvalue(x)
    return [ exp(x[2]-x[1]) - 2,
          x[1]*x[2] + x[3],
          x[2]*x[3] + x[1]^2 - x[2]
        ]
end

Here is a function that computes the Jacobian matrix.

In [ ]:
function nljac(x)
    J = zeros(3,3)
    J[1,:] = [-exp(x[2]-x[1]), exp(x[2]-x[1]), 0]
    J[2,:] = [x[2], x[1], 1]
    J[3,:] = [2*x[1], x[3]-1, x[2]]
    return J
end

(These functions could be written as separate files, or embedded within another function as we have done here.) Our initial guess at a root is the origin.

In [ ]:
x = [0,0,0]

We need the value (residual) of the nonlinear function, and its Jacobian, at this value for $\mathbf{x}$. 

In [ ]:
@show f = nlvalue(x);
@show J = nljac(x);

We solve for the Newton step and compute the new estimate. 

In [ ]:
s = -(J\f)
x = [x x[:,1]+s]

Here is the new residual.

In [ ]:
nlvalue(x[:,2])

We don't seem to be especially close to a root. Let's iterate a few more times. 

In [ ]:
for n = 2:7
    f = [f nlvalue(x[:,n])]
    s = -( nljac(x[:,n]) \ f[:,n] )
    x = [x x[:,n]+s]
end

We find the infinity norm of the residuals. 

In [ ]:
residual_norm = maximum(abs.(f),dims=1)   # max in dimension 1

We don't know an exact answer, so we will take the last computed value as its surrogate. 

In [ ]:
r = x[:,end]
x = x[:,1:end-1]

The following will subtract r from every column of x.

In [ ]:
e = x .- r

Now we take infinity norms and check for quadratic convergence. 

In [ ]:
errs = maximum(abs.(e),dims=1)
ratios = @. log(errs[2:end]) / log(errs[1:end-1])

For a brief time, we see ratios around 2, but then the limitation of double precision makes it impossible for the doubling to continue. 